In [1]:
usuario = r'gfloress'

ruta_mkp = r'MARKETPLACE PERÚ - Documents'
ruta_hd = r'PU - Transporte HomeDelivery - SX'

In [2]:
import pyarrow
import fastparquet

In [3]:
# Importar librerías de funciones
import os # Para trabajar con rutas
import glob # Para jalar todo los archivos en una carpeta
import warnings # Para evitar que salgan errores en formatos de archivo (no altera el producto)
import numpy as np # Para operaciones matemáticas
import pandas as pd # Para transformar la data
from pandas import ExcelWriter # Para exportar tabla a Excel

# Opciones
warnings.filterwarnings(action='ignore') # Eliminar mensajes de warning (no elimina mensajes de error)
pd.set_option('display.expand_frame_repr', False)
#pd.options.display.float_format = '{:.2f}'.format # Suprimir notación científica para números

# Rutas base
ruta_base_mkp = r'C:\\Users\\' + usuario + r'\\Falabella\\' + ruta_mkp
ruta_base_hd = r'C:\\Users\\' + usuario + r'\\Falabella\\' + ruta_hd

ruta_planificacion_sx = ruta_base_mkp + r'\\2. Operaciones\\6. F.com\\DataHD\\Historico Planificacion'
ruta_ing_no_planif_sx = ruta_base_mkp + r'\\2. Operaciones\\6. F.com\\DataHD\\Historico Ingreso No Planificado'

ruta_planificacion = ruta_base_hd + r'\\Historico Planificacion'
ruta_ing_no_planif = ruta_base_hd + r'\\Historico Ingreso No Planificado'

In [4]:
planificacion_files = glob.glob(os.path.join(ruta_planificacion, "[0-9]*.csv"))
planificacion = pd.DataFrame()
x = pd.DataFrame()
for i in range(len(planificacion_files)):
    x = pd.read_csv(planificacion_files[i], sep=",", encoding='latin-1', dtype={'rastreo': str})
    planificacion = pd.concat([planificacion,x], axis=0)

ing_no_planif_files = glob.glob(os.path.join(ruta_ing_no_planif, "[0-9]*.csv"))
ing_no_planif = pd.DataFrame()
x = pd.DataFrame()
for i in range(len(ing_no_planif_files)):
    x = pd.read_csv(ing_no_planif_files[i], sep=",", encoding='latin-1', dtype={'Rastreo': str})
    ing_no_planif = pd.concat([ing_no_planif,x], axis=0)

In [5]:
planificacion = (
    planificacion
    .drop(['supplier_name','supplier_id','ruc'], axis=1)
    .assign(**{
        'Fecha Planificación': lambda x: pd.to_datetime(x['Fecha Planificación'], infer_datetime_format=True, errors='coerce').dt.date,
        'rastreo': lambda x: x['rastreo'].astype(str).str.replace('(\.0$)', ''),
        })
    .sort_values(['Fecha Planificación'])
)

ing_no_planif = (
    ing_no_planif
    .assign(Rastreo=lambda df: df['Rastreo'].apply(lambda x: f"{x:.0f}" if isinstance(x, float) else str(x)))
    .assign(**{
        'Fecha Registro': lambda x: pd.to_datetime(x['Fecha Registro'], infer_datetime_format=True, errors='coerce').dt.date,
        'Rastreo': lambda x: x['Rastreo'].astype(str).str.replace('(\.0$)', ''),
        })
    .sort_values(['Fecha Registro'])
)

In [11]:
enero = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-01-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-02-01').date()) 
        ]

febrero = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-02-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-03-01').date()) 
        ]

marzo = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-03-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-04-01').date()) 
        ]

abril = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-04-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-05-01').date()) 
        ]

mayo = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-05-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-06-01').date()) 
        ]

junio = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-06-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-07-01').date()) 
        ]

In [12]:
enero.to_parquet(os.path.join(ruta_planificacion_sx,'2024_01.parquet'), index=False)
febrero.to_parquet(os.path.join(ruta_planificacion_sx,'2024_02.parquet'), index=False)
marzo.to_parquet(os.path.join(ruta_planificacion_sx,'2024_03.parquet'), index=False)
abril.to_parquet(os.path.join(ruta_planificacion_sx,'2024_04.parquet'), index=False)
mayo.to_parquet(os.path.join(ruta_planificacion_sx,'2024_05.parquet'), index=False)
junio.to_parquet(os.path.join(ruta_planificacion_sx,'2024_06.parquet'), index=False)

In [8]:
enero = ing_no_planif[
            (ing_no_planif['Fecha Registro']>=pd.to_datetime('2024-01-01').date()) & (ing_no_planif['Fecha Registro']<pd.to_datetime('2024-02-01').date()) 
        ]

febrero = ing_no_planif[
            (ing_no_planif['Fecha Registro']>=pd.to_datetime('2024-02-01').date()) & (ing_no_planif['Fecha Registro']<pd.to_datetime('2024-03-01').date()) 
        ]

marzo = ing_no_planif[
            (ing_no_planif['Fecha Registro']>=pd.to_datetime('2024-03-01').date()) & (ing_no_planif['Fecha Registro']<pd.to_datetime('2024-04-01').date()) 
        ]

abril = ing_no_planif[
            (ing_no_planif['Fecha Registro']>=pd.to_datetime('2024-04-01').date()) & (ing_no_planif['Fecha Registro']<pd.to_datetime('2024-05-01').date()) 
        ]

mayo = ing_no_planif[
            (ing_no_planif['Fecha Registro']>=pd.to_datetime('2024-05-01').date()) & (ing_no_planif['Fecha Registro']<pd.to_datetime('2024-06-01').date()) 
        ]

junio = ing_no_planif[
            (ing_no_planif['Fecha Registro']>=pd.to_datetime('2024-06-01').date()) & (ing_no_planif['Fecha Registro']<pd.to_datetime('2024-07-01').date()) 
        ]

In [9]:
enero.to_parquet(os.path.join(ruta_ing_no_planif_sx,'2024_01.parquet'), index=False)
febrero.to_parquet(os.path.join(ruta_ing_no_planif_sx,'2024_02.parquet'), index=False)
marzo.to_parquet(os.path.join(ruta_ing_no_planif_sx,'2024_03.parquet'), index=False)
abril.to_parquet(os.path.join(ruta_ing_no_planif_sx,'2024_04.parquet'), index=False)
mayo.to_parquet(os.path.join(ruta_ing_no_planif_sx,'2024_05.parquet'), index=False)
junio.to_parquet(os.path.join(ruta_ing_no_planif_sx,'2024_06.parquet'), index=False)

In [10]:
enero = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-01-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-02-01').date()) 
        ]

febrero = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-02-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-03-01').date()) 
        ]

marzo = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-03-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-04-01').date()) 
        ]

abril = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-04-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-05-01').date()) 
        ]

mayo = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-05-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-06-01').date()) 
        ]

junio = planificacion[
            (planificacion['Fecha Planificación']>=pd.to_datetime('2024-06-01').date()) & (planificacion['Fecha Planificación']<pd.to_datetime('2024-07-01').date()) 
        ]